# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [1]:
import pandas as pd
import numpy as np

In [2]:
flights = pd.read_csv('flights_rand.csv')

In [3]:
flights1 = flights.drop(['dest', 'dest_city_name','origin', 'origin_city_name', 'mkt_unique_carrier', 'branded_code_share', 'carrier_delay','weather_delay','nas_delay','security_delay', 'late_aircraft_delay', 'first_dep_time', 'total_add_gtime','longest_add_gtime','cancelled', 'cancellation_code', 'diverted', 'dup', 'flights', 'no_name'], axis = 1)

In [4]:
dep_time_mode = flights1['dep_time'].mode()
arr_time_mode = flights1['arr_time'].mode()
wheel_off_mode = flights1['wheels_off'].mode()
wheel_on_mode = flights1['wheels_on'].mode()
tail_number_mode = flights1['tail_num'].mode()

In [5]:
flights1['wheels_off'] = flights1['wheels_off'].fillna(int(wheel_off_mode[0]))
flights1['arr_time'] = flights1['arr_time'].fillna(int(arr_time_mode))
flights1['dep_time'] = flights1['dep_time'].fillna(int(dep_time_mode))
flights1['wheels_on'] = flights1['wheels_on'].fillna(int(wheel_on_mode))
flights1['tail_num'] = flights1['tail_num'].fillna(str(tail_number_mode[0]))

In [6]:
flights1['dep_delay'] = flights1['dep_delay'].fillna(flights1['dep_delay'].mean())
flights1['taxi_in'] = flights1['taxi_in'].fillna(flights1['taxi_in'].mean())
flights1['arr_delay'] = flights1['arr_delay'].fillna(flights1['arr_delay'].mean())
flights1['actual_elapsed_time'] = flights1['actual_elapsed_time'].fillna(flights1['actual_elapsed_time'].mean())
flights1['air_time'] = flights1['air_time'].fillna(flights1['air_time'].mean())
flights1['taxi_out'] = flights1['taxi_out'].fillna(flights1['taxi_out'].mean())

* adding month, day of the week, day of the month

In [24]:
flights1['fl_date'] = pd.to_datetime(flights1['fl_date'], errors='coerce')
flights1['month'] = flights1['fl_date'].dt.month
flights1['day_of_week'] = flights1['fl_date'].dt.dayofweek
flights1['day_of_month'] = flights1['fl_date'].dt.day
flights1['year'] = flights1['fl_date'].dt.year

In [29]:
flights1 = flights1.drop('fl_date', axis = 1)

* get_dummies categorical features

In [8]:
#cat_features = flights1[['mkt_carrier', 'op_unique_carrier', 'tail_num', 'op_carrier_fl_num', 'origin_airport_id', 'dest_airport_id']]

In [9]:
#df_dummy = pd.get_dummies(cat_features)

In [10]:
#df_dum = pd.get_dummies(flights1['dest_airport_id'])

* make the types categories

In [37]:
flights1["mkt_carrier"] = flights1["mkt_carrier"].astype("category")
flights1["op_unique_carrier"] = flights1["op_unique_carrier"].astype("category")
flights1["tail_num"] = flights1["tail_num"].astype("category")
flights1["op_carrier_fl_num"] = flights1["op_carrier_fl_num"].astype("category")
flights1["origin_airport_id"] = flights1["origin_airport_id"].astype("category")
flights1["dest_airport_id"] = flights1["dest_airport_id"].astype("category")
flights1["mkt_carrier_fl_num"] = flights1["mkt_carrier_fl_num"].astype("category")

In [31]:
flights1

,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,dep_delay,...,arr_time,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,distance,month,day_of_week,day_of_month,year
0,DL,5799,CP,N615CZ,5799,14869,12280,1100,1056.0,-4.0,...,1202.0,-2.0,64.0,66.0,43.0,188.0,7,4,27,2018
1,UA,6129,YV,N88325,6129,12266,11540,955,1016.0,21.0,...,1111.0,20.0,116.0,115.0,91.0,667.0,10,0,8,2018
2,WN,809,WN,N7841A,809,13232,11433,1640,1638.0,-2.0,...,1831.0,-19.0,70.0,53.0,36.0,228.0,10,2,31,2018
3,AA,1830,AA,N556UW,1830,12889,11057,750,750.0,0.0,...,1506.0,-2.0,258.0,256.0,221.0,1916.0,10,1,15,2019
4,AS,3300,OO,N176SY,3300,14831,10423,938,932.0,-6.0,...,1458.0,-16.0,216.0,206.0,187.0,1476.0,6,1,19,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,UA,6276,YV,N88335,6276,12266,13158,1445,1435.0,-10.0,...,1608.0,-11.0,94.0,93.0,69.0,429.0,1,5,12,2019
9996,AS,2497,QX,N422QX,2497,15023,12892,1400,1400.0,0.0,...,1528.0,-4.0,92.0,88.0,69.0,400.0,11,4,2,2018
9997,AA,2076,AA,N825AW,2076,14107,12339,855,904.0,9.0,...,1518.0,-9.0,212.0,194.0,173.0,1488.0,5,0,20,2019
9998,WN,3107,WN,N202WN,3107,11278,10693,1650,1734.0,44.0,...,1813.0,33.0,110.0,99.0,80.0,562.0,8,6,4,2019


In [39]:
# ENCODE AIRPORTS AND TAILNUM
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
flights1['mkt_carrier'] = encoder.fit_transform(flights1[['mkt_carrier']])
flights1['mkt_carrier_fl_num'] = encoder.fit_transform(flights1[['mkt_carrier_fl_num']])
flights1['op_unique_carrier'] = encoder.fit_transform(flights1[['op_unique_carrier']])
flights1['tail_num'] = encoder.fit_transform(flights1[['tail_num']])
flights1['op_carrier_fl_num'] = encoder.fit_transform(flights1[['op_carrier_fl_num']])
flights1['origin_airport_id'] = encoder.fit_transform(flights1[['origin_airport_id']])
flights1['dest_airport_id'] = encoder.fit_transform(flights1[['dest_airport_id']])

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

In [ ]:
LDA VS PCA

* PCA testing

In [14]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
pca_test = PCA(n_components=5)
#principalComponents_test = pca_test.fit_transform(flights1)

### Modeling

Use different ML techniques to predict each problem.

- linear
- Naive Bayes
- Random Forest Regressor
- SVM classification
- XGBoost regresspr
- The ensemble of your own choice

* target and features

In [117]:
y = np.array(flights1.arr_delay)

In [118]:
y

array([ -2.,  20., -19., ...,  -9.,  33.,  -9.])

In [42]:
X_df = flights1.drop('arr_delay', axis = 1)

In [43]:
X = np.array(X_df)

In [114]:
y

array([0, 0, 0, ..., 0, 0, 0])

* Scaling

In [44]:
from sklearn.preprocessing import StandardScaler

In [46]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [47]:
y = (y - y.mean()) / y.std()

* train test split

In [49]:
import sklearn.model_selection as model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.90,test_size=0.10, random_state=101, shuffle = True)

In [107]:
type(X_train)

numpy.ndarray

In [113]:
type(y_train)

numpy.ndarray

In [ ]:
y_train

In [67]:
from sklearn.metrics import r2_score

#### linear

In [89]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

In [90]:
#logistic regression
linear_model = LinearRegression()

In [91]:
linear_model.fit(X_train, y_train)

LinearRegression()

In [92]:
y_pred_linear = linear_model.predict(X_test)

In [94]:
r2_score(y_test, y_pred_linear)

0.9878448209808578

#### Naives Bayes, GaussianNB Naive Bayes

In [110]:
from sklearn.naive_bayes import GaussianNB

In [111]:
NV_Gauss_model = GaussianNB()

In [112]:
NV_Gauss_model.fit(X_train, y_train)

ValueError: Unknown label type: (array([-1.65141635e+00, -1.38684079e+00, -1.36648883e+00, -1.32578490e+00,
       -1.24437703e+00, -1.18332114e+00, -1.12226524e+00, -1.10191327e+00,
       -1.08156131e+00, -1.06120934e+00, -1.04085738e+00, -1.02050541e+00,
       -1.00015344e+00, -9.79801479e-01, -9.59449513e-01, -9.39097548e-01,
       -9.18745582e-01, -8.98393616e-01, -8.78041651e-01, -8.57689685e-01,
       -8.37337719e-01, -8.16985754e-01, -7.96633788e-01, -7.76281822e-01,
       -7.55929857e-01, -7.35577891e-01, -7.15225925e-01, -6.94873960e-01,
       -6.74521994e-01, -6.54170028e-01, -6.33818063e-01, -6.13466097e-01,
       -5.93114131e-01, -5.72762166e-01, -5.52410200e-01, -5.32058234e-01,
       -5.11706269e-01, -4.91354303e-01, -4.71002337e-01, -4.50650372e-01,
       -4.30298406e-01, -4.09946440e-01, -3.89594475e-01, -3.69242509e-01,
       -3.48890543e-01, -3.28538578e-01, -3.08186612e-01, -2.87834646e-01,
       -2.67482681e-01, -2.47130715e-01, -2.26778749e-01, -2.064267

In [ ]:
y_pred_NV_multi = NV_multi_grid.predict(y_test)

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.